### DPS Test 방법 : 아래 각 코드를 터미널에서 입력해야함
0. 가상환경 말고 그냥 처음 터미널에서, `~/legal_llm/dps` 경로 상에서 아래 코드 실행해야함
1. `python bin/sparkapp.py korean_job --config_path=./configs/dedup_job_case_law.yaml` : yaml에서 config 바꿔줘야 함
2. `wc -l /home/seonghee_hong/legal_llm/dps/legal-project-data/DOMAIN_First_Dedup_CASE_LAW_0.8_16/part-00000` : 길이 확인

In [1]:
import os
import sys
import json
import random

# 필요한 경로 추가
base_path = '/home/seonghee_hong/legal_llm/dps'
sys.path.append(base_path)
sys.path.append(f'{base_path}/dps/spark')
sys.path.append(f'{base_path}/dps/spark/jobs')

from jobs.korean_job import *
from jobs.dedup_job import *
from prep.dedup_prep import *

In [2]:
sample_text = "이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다."
print("Before Preprocess:", sample_text)

config_path = '/home/seonghee_hong/legal_llm/dps/configs/dedup_job_non_case_law.yaml'

with open(config_path) as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)

# 위 필터 함수들을 한 번에 적용하는 함수 # ! 단순 테스트용임. 실제로는 사용 X
def total_filter(text):
    return bad_words_filter(text) and doc_len_filter(text, conf["min_doc_len"], conf["max_doc_len"]) and mean_word_len_filter(text, conf["min_mean_word_len"], conf["max_mean_word_len"]) and symbol_to_word_ratio_filter(text, conf["symbol_to_word_ratio"]) and bullet_ellipsis_filter(text, conf["bullet_point_ratio"], conf["ellipsis_ratio"]) and korean_word_ratio_filter(text, conf["korean_word_ratio"])

# 전처리 함수들을 한 번에 적용하는 함수 # ! 단순 테스트용임. 실제로는 사용 X
def total_preprocess(text): 
    if total_filter(text):
        return preprocess_text(make_compat(text))
    else:
        return None

total_preprocess_result = total_preprocess(sample_text)
print("Total preprocess result:", total_preprocess_result)

Before Preprocess: 이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다.
Total preprocess result: 이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다.


In [4]:
# 전처리 전 데이터 불러오기
file_path = "/home/seonghee_hong/legal_llm/dps/legal-project-data/NON-CASE-LAW-sample/sampled.jsonl"
with open(file_path, "r") as f:
    data = f.readlines()
    bf_data = [json.loads(line) for line in data]

bf_texts = [d["text"] for d in bf_data]

In [7]:
idx = random.randint(0, len(bf_texts))
print(f"Random index: {idx}")

# 전처리 전 데이터
print(f"=== Before preprocessing  === \n {bf_texts[idx]}")

# 전처리 후 데이터
print(f"=== After preprocessing === \n {total_preprocess(bf_texts[idx])}")

Random index: 2283
=== Before preprocessing  === 
 요약대상 원문 개정안들은 2022년 12월 31일자로 일몰될 예정인 조합 등 출자금·예탁금에 대한 비과세의 적용기한을 연장함으로써 해당 조합 등의 조합원, 준조합원 또는 회원(이하 “조합원등”이라 함)인 농어민·서민의 세부담을 경감하려는 것임.
 현행법에 따라 조합 등 출자금·예탁금에서 발생하는 이자·배당소득에 대해서는 2022년 12월 31일까지 비과세 하고, 2023년에는 5%, 2024년 이후에는 9%로 저율 분리과세 할 예정임.
 구체적으로, 조합 등 출자금에 대해서 김선교·윤영석·김주영의원안은 2024년 12월 31일까지(2년간), 배준영·주철현·김수흥·류성걸의원안 및 정부안은 2025년 12월 31일까지(3년간), 이원택의원안은 2026년 12월 31일까지(4년간), 윤재갑·김경협·윤준병·장동혁·김희곤의원안은 2027년 12월 31일까지(5년간), 안호영의원안은 2032년 12월 31일까지(10년간) 비과세 기한을 연장하려는 것이고,
 조합 등 예탁금에 대해서는 정점식·김선교·윤영석·김주영의원안은 2024년 12월 31일까지(2년간), 배준영·주철현·김수흥·류성걸의원안 및 정부안은 2025년 12월 31일까지(3년간), 이원택의원안은 2026년 12월 31일까지(4년간), 김경협·윤준병·장동혁·김희곤의원안은 2027년 12월 31일까지(5년간), 안호영의원안은 2032년 12월 31일까지(10년간) 비과세 기한을 연장하려는 것임.
 조합 등 출자금·예탁금에 대한 비과세는 농어민·서민의 재산형성을 지원하고, 서민 금융지원업무를 수행하는 상호금융의 재무구조 개선 및 원활한 자금조달을 지원하기 위한 목적으로 1976년부터 시행되고 있음. 조합 등에 대한 조합원등 1명당 1천만원 이하의 출자금에 대한 배당소득과 사업 이용 실적에 따른 배당소득(이용고배당소득), 20세 이상 조합원등 1명당 3천만원 이하의 예탁금에 대한 이자소득에 대하여는 2022년까지 소득세를 부과하

## NON-CASE-LAW 수정 관련 메모
#### 현재 코드에서 수정해야할 부분
- 별도로 없는 거 같음
#### NONE-CASE-LAW 데이터 제거 대상
- 별도로 없는 거 같음
#### NONE-CASE-LAW 데이터 변형 대상 -> korean_prep.py에서 replace_korean_pii 관련 파트 수정
- "안성포 전남대 로스쿨 교수", "김오수 검찰총장", "이사장 이순국", "민형배의원" 등 실명이 등장하지만, 공통되는 패턴이 없음

## CASE-LAW 수정 관련 메모
#### 현재 코드에서 수정해야할 부분
- "판결선고일 1997-09-09 " -> "판결선고일 <|acc_start|>1312-78-89<|acc_end|>"와 같이 변함. 즉, 일종의 Special Token이 추가되고, 숫자 자체가 변함
- 찾아보니까, korean_utils.py에서 line 240 ~ 242 : ACCOUNT_PATTERN 부분이 문제임 -> CASE-LAW에서는 아예 해당 코드는 주석 먹이고 적용해도 될 듯 (하지만 이러면, General과 Domain 별로 코드가 달라짐) => 준원 님과 논의 결과 Domain 데이터 돌릴 때만 잠시 주석 먹이는 방향으로 하기로 함

#### CASE-LAW 데이터 제거 대상
- " ~~~ 주문과 같이 판결한다.  판사 김용덕(재판장) 고진흥 임한아"  -> "판사 김용덕(재판장) ~ " => 완료

#### CASE-LAW 데이터 변형 대상 -> korean_prep.py에서 replace_korean_pii 부분 수정해야할 듯
- "~~~ 변호사 이수엽 ~~~" -> " ~~~ 변호사 XXX"
- " ~~~ 법무법인 도울 ~~~~ " -> " ~~~ 법무법인 XX ~~~"
- " ~~~~ 법무법인 케이씨엘 ~~~~ " -> " ~~~ 법무법인 XXXX ~~~~"
- " 【원    고】 주식회사 신영 (소송대리인 법무법인 광장 담당변호사 이환구 외 1인) " -> " 주식회사 XX (소송대리인 법무법인 XX 담당변호사 XXX 외 1인) "

In [19]:
import json

### 일차 전처리 완료 후 데이터 불러와서 확인
file_path = '/data/llmlaw/PREPROCESSSED/Domain/CASE-LAW/part-00000'

with open(file_path, "r") as f:
    data = f.readlines()
    af_data = [json.loads(line) for line in data]

af_texts = [d["text"] for d in af_data]

In [21]:
# 위 파일을 다시 jsonl 형태로 저장
with open("/data/llmlaw/PREPROCESSSED/Domain/CASE-LAW/preprocessed.jsonl", "w") as f:
    for line in af_data:
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

# 다시 불러와서 확인
file_path = "/data/llmlaw/PREPROCESSSED/Domain/CASE-LAW/preprocessed.jsonl"
with open(file_path, "r") as f:
    data = f.readlines()
    af_data = [json.loads(line) for line in data]

af_texts = [d["text"] for d in af_data]

In [18]:
import random

idx = random.randint(0, len(af_texts))
print(f"Random index: {idx}")
print(f"=== After preprocessing === \n {af_texts[idx]}")

Random index: 117500
=== After preprocessing === 
 판시사항 내용없음 판결요지 내용없음 참조조문 내용없음 참조판례 내용없음 판례내용 【원 고】 원고 (소송대리인 법무법인 XXXX 담당변호사 XXX 외 1인)【피 고】 육군참모총장 외 2인【변론종결】2017. 4. 20.【주 문】 1. 원고의 피고 육군종합군수학교장에 대한 소를 각하한다. 2. 원고의 피고 육군참모총장 및 피고 대한민국에 대한 각 청구를 모두 기각한다. 3. 소송비용은 원고가 부담한다. 【청구취지】 피고 육군참모총장이 2015. 9. 23. 원고에 대하여 한 퇴역(명예전역)명령이 유효함을 확인하고, 피고 육군종합군수학교장이 2016. 2. 1. 원고에 대하여 한 임용무효처분을 취소하며, 피고 대한민국은 원고가 정년전역 및 퇴역 대상자임을 확인한다. 【이 유】1. 처분의 경위 가. 원고는 1983. 1. 7. 육군에 입대하여 1983. 3. 28. 하사관후보생에 임명되었다가, 1983. 6. 18. 단기복무 하사관에 임용되어 복무하던 중 1986. 6. 1. 장기복무 하사관에 임용되었다. 나. 원고가 1985. 11. 1. 중사로, 1996. 4. 1. 상사로, 2010. 3. 1. 원사로 진급하여 육군에서 복무하던 중 군인사법 제53조의2 규정에 의한 명예전역을 신청함에 따라 피고 육군참모총장은 2015. 9. 23. 원고에 대하여 2015. 12. 31.부 명예전역을 명하였다. 다. 이후 피고 육군참모총장은 원고가 1982. 12. 30. 대구지방법원에서 1982. 7.~9.경의 폭력행위등처벌에관한법률위반 등의 범죄(이하 ‘종전 범죄’라 한다)사실로 징역 1년, 집행유예 3년의 형을 선고받았고, 그 무렵 항소기간 도과로 위 판결(이하 ‘종전 판결’이라고 한다)이 확정된 사실을 확인하였고, 이에 따라 피고 육군종합군수학교장은 2016. 1. 29. 원고에 대한 단기복무 하사관 임용명령을 무효로 하는 인사명령을 발령하였다. [인정근거] 다툼 없는 사실, 갑 제